<a href="https://colab.research.google.com/github/CAVASOL/aiffel_quest/blob/main/Exploration_quest/exploration_6/xp6_mlops.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Exploration 6. MLOps

**Goal**

* Basics of MLOps Theory
* Hyperparameter tuning with KerasTuner
* Deploy models with TFServing or TFLite

**Index**

    1. Definition of MLOps and components of an ML system
    2. TFX (TensorFlow Extended)
    3. KerasTuner
    4. TFServing & TFLite

### 1. Definition of MLOps and components of an ML system

[Kubeflow](https://www.kubeflow.org/docs/started/introduction/)  
[What is Airflow](https://airflow.apache.org/docs/apache-airflow/stable/index.html)  
[Hidden Technical Debt in Machine Learning Systems](https://proceedings.neurips.cc/paper_files/paper/2015/file/86df7dcfd896fcaf2674f757a2463eba-Paper.pdf)  

###  2. TFX (TensorFlow Extended)

[TFX is an end-to-end platform for deploying production ML pipelines](https://www.tensorflow.org/tfx)  

### 3. KerasTuner

[KerasTuner](https://keras.io/keras_tuner/)

In [ ]:
!mkdir ~/aiffel/mlops
!pip install keras-tuner

mkdir: cannot create directory ‘/aiffel/aiffel/mlops’: File exists


In [ ]:
import tensorflow as tf
import keras
import keras_tuner as kt
from sklearn.model_selection import train_test_split
import os

In [ ]:
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

In [ ]:
X_train = x_train.reshape(-1,28, 28, 1)
X_test = x_test.reshape(-1,28,28,1)
y_train = tf.keras.utils.to_categorical(y_train)
y_test = tf.keras.utils.to_categorical(y_test)

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size = 0.2)
print(X_train.shape)

(48000, 28, 28, 1)


In [ ]:
class DeepTuner(kt.Tuner):
    def run_trial(self, trial, X, y, validation_data, **fit_kwargs):
        model = self.hypermodel.build(trial.hyperparameters)
        model.fit(X, y, batch_size=trial.hyperparameters.Choice(
            'batch_size', [16, 32]), **fit_kwargs)


        X_val, y_val = validation_data
        eval_scores = model.evaluate(X_val, y_val)
        return {name: value for name, value in zip(
            model.metrics_names,
            eval_scores)}

In [ ]:
def build_model(hp):
    model = tf.keras.Sequential()
    model.add(tf.keras.Input(shape = X_train.shape[1:], name = 'inputs'))
    for i in range(hp.Int('num_layers', min_value=1, max_value=10)):
              model.add(tf.keras.layers.Conv2D(hp.Int(
                  'units_{i}'.format(i=i), min_value=32, max_value=128, step=5), (3,3),activation='relu'))
    model.add(tf.keras.layers.Flatten())
    for i in range(hp.Int('n_connections', 1, 3)):
        model.add(tf.keras.layers.Dense(hp.Choice(f'n_nodes',
                                  values=[32,64,128, 256]), activation = 'relu'))
    model.add(tf.keras.layers.Dense(10, activation='softmax', name = 'outputs'))
    model.compile(optimizer = 'adam',loss='categorical_crossentropy',
        metrics=['accuracy'])
    return model

In [ ]:
my_keras_tuner = DeepTuner(
    oracle=kt.oracles.BayesianOptimizationOracle(
        objective=kt.Objective('accuracy', 'max'),
        max_trials=10,
        seed=42),
    hypermodel=build_model,
    overwrite=True,
    project_name='my_keras_tuner')

my_keras_tuner.search(X_train, y_train, validation_data=(X_val, y_val), epochs=3)

Trial 10 Complete [00h 00m 46s]
accuracy: 0.9750833511352539

Best accuracy So Far: 0.984666645526886
Total elapsed time: 00h 08m 00s


In [ ]:
best_hps = my_keras_tuner.get_best_hyperparameters(num_trials=10)[0]
model = build_model(best_hps)
model.summary()

Model: "sequential_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_65 (Conv2D)           (None, 26, 26, 102)       1020      
_________________________________________________________________
conv2d_66 (Conv2D)           (None, 24, 24, 42)        38598     
_________________________________________________________________
conv2d_67 (Conv2D)           (None, 22, 22, 67)        25393     
_________________________________________________________________
conv2d_68 (Conv2D)           (None, 20, 20, 37)        22348     
_________________________________________________________________
conv2d_69 (Conv2D)           (None, 18, 18, 52)        17368     
_________________________________________________________________
flatten_11 (Flatten)         (None, 16848)             0         
_________________________________________________________________
dense_16 (Dense)             (None, 256)             

In [ ]:
model.fit(X_train, y_train, batch_size=32, epochs = 5)

Epoch 1/5
1500/1500 [==============================] - 11s 7ms/step - loss: 0.1891 - accuracy: 0.9543
Epoch 2/5
1500/1500 [==============================] - 10s 7ms/step - loss: 0.0498 - accuracy: 0.9848
Epoch 3/5
1500/1500 [==============================] - 10s 7ms/step - loss: 0.0351 - accuracy: 0.9892
Epoch 4/5
1500/1500 [==============================] - 10s 7ms/step - loss: 0.0271 - accuracy: 0.9915
Epoch 5/5
1500/1500 [==============================] - 10s 7ms/step - loss: 0.0190 - accuracy: 0.9936


In [ ]:
model.evaluate(X_test, y_test)

313/313 [==============================] - 1s 4ms/step - loss: 0.0462 - accuracy: 0.9866


[0.0461813360452652, 0.9865999817848206]

In [ ]:
save_path = os.getenv('HOME') + '/aiffel/mlops/best_model/1'
fname = os.path.join(save_path, 'model')
model.save(fname)

INFO:tensorflow:Assets written to: /aiffel/aiffel/mlops/best_model/1/model/assets


### 4. TFServing & TFLite

In [ ]:
load_path = os.getenv('HOME') + '/aiffel/mlops/best_model/1/model'
best_model = tf.keras.models.load_model(load_path)

In [ ]:
best_model.summary()

Model: "sequential_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_65 (Conv2D)           (None, 26, 26, 102)       1020      
_________________________________________________________________
conv2d_66 (Conv2D)           (None, 24, 24, 42)        38598     
_________________________________________________________________
conv2d_67 (Conv2D)           (None, 22, 22, 67)        25393     
_________________________________________________________________
conv2d_68 (Conv2D)           (None, 20, 20, 37)        22348     
_________________________________________________________________
conv2d_69 (Conv2D)           (None, 18, 18, 52)        17368     
_________________________________________________________________
flatten_11 (Flatten)         (None, 16848)             0         
_________________________________________________________________
dense_16 (Dense)             (None, 256)             

In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

INFO:tensorflow:Assets written to: /tmp/tmp2a5deyga/assets


In [ ]:
with open('model.tflite', 'wb') as f:
  f.write(tflite_model)

In [ ]:
interpreter = tf.lite.Interpreter(model_content=tflite_model)

signatures = interpreter.get_signature_list()
print(signatures)

{}


In [ ]:
# classify_lite = interpreter.get_signature_runner('serving_default')
# classify_lite